In [2]:
import csv
import pandas as pd
import json
import numpy as np
from pymongo import MongoClient

csv_file="/home/rtx2070s/exp/deep-latent-sequence-model/data/yelp_long/yelp_academic_dataset_review.json"
#too heavy
# with open(csv_file, 'r') as f:
#     js_file=json.load(f)
#     print(js_file)


In [2]:
#use too much memory ,careful!!!
mongo_client=MongoClient('localhost',27017)
dblist=mongo_client.list_database_names()

db = mongo_client.yelpreview
col=db.yelp_academic_dataset_review


mongo_docs = col.find()
#easier implement
entries=list(mongo_docs)
entries[:5]
df=pd.DataFrame(entries)
print(df.head())

                        _id               review_id                 user_id  \
0  6225d46862b5a7859ec3e87c  NDhkzczKjLshODbqDoNLSg  eC5evKn1TWDyHCyQAwguUw   
1  6225d46862b5a7859ec3e87d  2l_TDrQ7p-5tANOyiOlkLQ  LWUnzwK0ILquLLZcHHE1Mw   
2  6225d46862b5a7859ec3e87e  8bFej1QE5LXp4O05qjGqXA  YoVfDbnISlW0f7abNQACIg   
3  6225d46862b5a7859ec3e87f  J4a2TuhDasjn2k3wWtHZnQ  RNm_RWkcd02Li2mKPRe7Eg   
4  6225d46862b5a7859ec3e880  T5fAqjjFooT4V0OeZyuk1w  SFQ1jcnGguO0LYWnbbftAA   

              business_id  stars  useful  funny  cool  \
0  _sS2LBIGNT5NQb6PD1Vtjw    5.0       0      0     0   
1  mD-A9KOWADXvfrZfwDs-jw    4.0       1      0     0   
2  RA4V8pr014UyUbDvI-LW2A    4.0       1      0     0   
3  xGXzsc-hzam-VArK6eTvtw    1.0       2      0     0   
4  0AzLzHfOJgL7ROwhdww2ew    2.0       1      1     1   

                                                text                 date  
0  I called WVM on the recommendation of a couple...  2013-05-28 20:38:06  
1  I am definitely a fan of Sp

In [8]:
df['text']

0          I called WVM on the recommendation of a couple...
1          I am definitely a fan of Sports Authority.  Th...
2          This store is pretty good. Not as great as Wal...
3          This place used to be a cool, chill place. Now...
4          I've stayed at many Marriott and Renaissance M...
                                 ...                        
8635398    This was absolutely the most egregious display...
8635399    This is so highly rated for a reason. If you'r...
8635400    This used to be my car wash of choice. They di...
8635401    This place is a wonderland of Asian delight. I...
8635402    In December, I called in (as a returning custo...
Name: text, Length: 8635403, dtype: object

In [9]:
df_ts=pd.DataFrame(df,columns=['stars','text'])

In [10]:
df_ts

,stars,text
0,5.0,I called WVM on the recommendation of a couple...
1,4.0,I am definitely a fan of Sports Authority. Th...
2,4.0,This store is pretty good. Not as great as Wal...
3,1.0,"This place used to be a cool, chill place. Now..."
4,2.0,I've stayed at many Marriott and Renaissance M...
...,...,...
8635398,1.0,This was absolutely the most egregious display...
8635399,5.0,This is so highly rated for a reason. If you'r...
8635400,2.0,This used to be my car wash of choice. They di...
8635401,5.0,This place is a wonderland of Asian delight. I...


In [13]:
neg_df=df_ts.loc[df_ts['stars']==1.0]
neg_df

,stars,text
3,1.0,"This place used to be a cool, chill place. Now..."
6,1.0,"They NEVER seem to get our \norder correct, se..."
9,1.0,I wish I could give them zero stars. The call ...
11,1.0,Great coffee and pastries. Baristas are excell...
26,1.0,Almost desolate restaurant and dingy evironmen...
...,...,...
8635359,1.0,I have been a loyal customer of this pizza pla...
8635370,1.0,Wouldn't even let a pregnant woman use the bat...
8635396,1.0,This guy is a moron that will stalk and harass...
8635398,1.0,This was absolutely the most egregious display...


In [14]:
neg_df['label']=0
neg_df

/tmp/ipykernel_14410/969301467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_df['label']=0


,stars,text,label
3,1.0,"This place used to be a cool, chill place. Now...",0
6,1.0,"They NEVER seem to get our \norder correct, se...",0
9,1.0,I wish I could give them zero stars. The call ...,0
11,1.0,Great coffee and pastries. Baristas are excell...,0
26,1.0,Almost desolate restaurant and dingy evironmen...,0
...,...,...,...
8635359,1.0,I have been a loyal customer of this pizza pla...,0
8635370,1.0,Wouldn't even let a pregnant woman use the bat...,0
8635396,1.0,This guy is a moron that will stalk and harass...,0
8635398,1.0,This was absolutely the most egregious display...,0


In [16]:
pos_df=df_ts.loc[df_ts['stars']==5.0]
pos_df

,stars,text
0,5.0,I called WVM on the recommendation of a couple...
5,5.0,I work in the Pru and this is the most afforda...
7,5.0,This is a five-star restaurant if ever I have ...
15,5.0,I loved everything about this place. I've only...
18,5.0,Definitely 5 stars for the donuts. Our family...
...,...,...
8635394,5.0,Best pizza in the neighborhood!!! Love the thi...
8635395,5.0,"Yummy, great chew on the bagel\nFriendly staff..."
8635397,5.0,Yes please! We had the Arepas falafel style an...
8635399,5.0,This is so highly rated for a reason. If you'r...


In [17]:
pos_df['label']=1

/tmp/ipykernel_14410/2336564515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df['label']=1


In [24]:
df_ts.loc[df_ts['stars']==5.0,'label']=1
df_ts.loc[df_ts['stars']==1.0,'label']=0
df_ts

,stars,text,label
0,5.0,I called WVM on the recommendation of a couple...,1.0
1,4.0,I am definitely a fan of Sports Authority. Th...,NaN
2,4.0,This store is pretty good. Not as great as Wal...,NaN
3,1.0,"This place used to be a cool, chill place. Now...",0.0
4,2.0,I've stayed at many Marriott and Renaissance M...,NaN
...,...,...,...
8635398,1.0,This was absolutely the most egregious display...,0.0
8635399,5.0,This is so highly rated for a reason. If you'r...,1.0
8635400,2.0,This used to be my car wash of choice. They di...,NaN
8635401,5.0,This place is a wonderland of Asian delight. I...,1.0


In [34]:
senti_df=df_ts.loc[df_ts['label'].notnull()]

In [35]:
senti_df=senti_df.sort_values(by='stars')

In [37]:
senti_head=senti_df.head(10)

In [43]:
senti_df.to_csv('senti_csv' ,columns=['stars','text','label'],index=False,)

In [ ]:
#start load data from here

In [1]:
import csv
import pandas as pd
import json
import numpy as np

In [2]:
senti_df=pd.read_csv('senti_csv')

In [3]:
senti_df

,stars,text,label
0,1.0,"Parents, teachers and students BEWARE before b...",0.0
1,1.0,Do NOT use these guys. They looked at our wash...,0.0
2,1.0,"""Women caring for women"" is a total joke when ...",0.0
3,1.0,Sloppy work sold me headers installed had a le...,0.0
4,1.0,This review is based on the comedy club itself...,0.0
...,...,...,...
5077327,5.0,Yaaaaay! Just passed my road test. Ask for Yur...,1.0
5077328,5.0,This is the best pizza around.The sauce is the...,1.0
5077329,5.0,First time at Black Wing and had a great time....,1.0
5077330,5.0,We came for lunch and sat on the beautiful pat...,1.0


In [4]:
senti_df['text'][4]

"This review is based on the comedy club itself and not the comedians. \n\nIf you want to waste $70 for a few chuckles, bad food, and parking, make it an Uptown Comedy Club night. Like 90% of places in Atlanta, there is no free parking near the club. It's $15 for valet, but there are also a few places down the street that charge $10. You may be lucky enough to find a parking spot on the street for free, but every time I've found an empty street space in Atlanta, there's always been somebody waiting for me to get out of my car so they can beg me for a couple of bucks. How do I get into the parking spot business? That must be the best way to become a millionaire in Atlanta. \nAnyway, be aware that there is a two item minimum once you're inside the club. There is no such thing as free water. You have to buy $6 designer water. There are a few food items on the menu and a few drinks. My drink tasted like it had been sitting in a fridge for a week soaking up the tastes of other foods. They p

In [5]:
senti_df=senti_df.loc[senti_df['text'].notnull()]

In [6]:
senti_df

,stars,text,label
0,1.0,"Parents, teachers and students BEWARE before b...",0.0
1,1.0,Do NOT use these guys. They looked at our wash...,0.0
2,1.0,"""Women caring for women"" is a total joke when ...",0.0
3,1.0,Sloppy work sold me headers installed had a le...,0.0
4,1.0,This review is based on the comedy club itself...,0.0
...,...,...,...
5077327,5.0,Yaaaaay! Just passed my road test. Ask for Yur...,1.0
5077328,5.0,This is the best pizza around.The sauce is the...,1.0
5077329,5.0,First time at Black Wing and had a great time....,1.0
5077330,5.0,We came for lunch and sat on the beautiful pat...,1.0


In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(senti_df, test_size=0.2,random_state=4522)
test,dev = train_test_split(test, test_size=0.5,random_state=4522)


In [10]:
train=train.sort_values(by='stars')
dev=dev.sort_values(by='stars')
test=test.sort_values(by='stars')



In [11]:
train

,stars,text,label
1213589,1.0,Can't believe I just paid 12 dollars for this ...,0.0
239355,1.0,So we came in for Wing night on a Monday and l...,0.0
900328,1.0,"Don't go to this store. The manager is bitch, ...",0.0
135973,1.0,I had used this company in the past & they wer...,0.0
391471,1.0,I am ecstatic that I can finally be closer to ...,0.0
...,...,...,...
2666686,5.0,Been here numerous times and just love the foo...,1.0
2455306,5.0,I'm glad something finally took over the empty...,1.0
4885162,5.0,This place is stupid good. We came for lunch w...,1.0
4075150,5.0,I was hesitant at first with the crowds but Ju...,1.0


In [12]:
dev

,stars,text,label
113110,1.0,I really hate to give bad reviews but Town Fai...,0.0
74713,1.0,Our A/C unit went on the fritz a few days ago....,0.0
192903,1.0,This place...I gave it one star because the fo...,0.0
881311,1.0,My sister's wedding is in two weeks and her br...,0.0
500586,1.0,They turned us away and said they were booked ...,0.0
...,...,...,...
3379255,5.0,Great variety of southern home cooking! Large...,1.0
3764775,5.0,I love this pizza! The prices are good and the...,1.0
1582452,5.0,Fantastic staff. Extremely friendly owner. Fan...,1.0
1906948,5.0,I've needed to do this for a while. I've relie...,1.0


In [7]:
import re
import os

In [8]:
import string
regular_punct = list(string.punctuation)

def spacing_punctuation(text, regular_punct):
    #standard punction
    text=re.sub('[!]+','!',text)
    text=re.sub('[.]+','.',text)
    for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, " "+f'{punc}')
    return text.strip()



def clean_txt(txt):
    #remove non-English
    
    
    #astro lookup
    contractions = {
    "ain't": "am not / are not",
    "aren't": "are not ",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I had",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    for word in txt.split():
        if word.lower() in contractions:
            txt = txt.replace(word, contractions[word.lower()])
    #remove space
    txt_list = txt.split()
    txt =" ".join(txt_list)
#     #join_words
#     txt = " ".join(re.findall('[A-Z][^A-Z]*', txt))
    #punction
    clean_txt=spacing_punctuation(txt, regular_punct)
    #num
    clean_txt=re.sub(r'[0-9]+:[0-9]+','_num_',clean_txt)
    clean_txt=re.sub(r'[0-9]+/[0-9]+','_num_',clean_txt)
    clean_txt=re.sub('[0-9]+,','_num_',clean_txt)
    return clean_txt

In [9]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def filtertext(s):
    
    filter_set=['http','[*]+']
    count=0
    for str in filter_set:
        pattern=re.compile(str)
        
        if re.search(pattern,s):
            count+=1
    if count>0:
        s=''
    return s

def extract_text(text):
    text_remove=re.sub('\n+', '\n', text)

    text_split=text_remove.split('\n')
    text_fin=text_split[0].strip().lower()+' '+text_split[-1].strip().lower()
#         print(text_fin)
#remove non-english
    if isEnglish(text_fin):
        text_fin=clean_txt(text_fin)
        
    else: 
#         print(text_fin"\n")
        text_fin=''
#fileter http sen
    text_fin=filtertext(text_fin)
    return text_fin
    

In [10]:
def write_file(df,tf_name_1,tf_name_2,max_length=50,label_name='label'):
    save_dir='./'
    ftrain_t=open(os.path.join(save_dir,tf_name_1),'w')
    ftrain_a=open(os.path.join(save_dir,tf_name_2),'w')
    sen=[]
    length_counts=[]
    labels=[]
    dropped=0
    counts=0
    for idx,data in df.iterrows():
        line=data['text']
        line=extract_text(line)
#         print(line)
        split_line=line.split()
        line=line+'\n'
        lc=len(split_line)
        lb=data[label_name]
        length_counts.append(lc)
        sen.append(line)
        labels.append(lb)
        counts +=1
        
        if len(split_line)<1:
            dropped +=1
#             print("sentence:",idx)
            continue
        
        if max_length:
            if len(split_line)>max_length:
                dropped +=1
                continue
        
        
        ftrain_t.write(line)
        if lb==0:
            ftrain_a.write('negative\n')
        else:
            ftrain_a.write('positive\n')

    return length_counts,sen,labels,dropped,counts
        
        
    
    

In [18]:
#split the txt data by length
def split_by_length(allt_name,alla_name,tf_name_st,tf_name_sa,tf_name_lt,tf_name_la,split_length=20,):
    save_dir='./'
    ftrain_st=open(os.path.join(save_dir,tf_name_st),'w')
    ftrain_sa=open(os.path.join(save_dir,tf_name_sa),'w')
    ftrain_lt=open(os.path.join(save_dir,tf_name_lt),'w')
    ftrain_la=open(os.path.join(save_dir,tf_name_la),'w')
    ftrain_at=open(os.path.join(save_dir,allt_name),'r')
    ftrain_aa=open(os.path.join(save_dir,alla_name),'r')
    lines=ftrain_at.readlines()
    attrs=ftrain_aa.readlines()
    for line,attr in zip(lines,attrs):
        split_line=line.strip().split()
        sen_len=len(split_line)
        if sen_len <=split_length:
            ftrain_st.write(line.strip()+'\n')
            ftrain_sa.write(attr.strip()+'\n')
        else:
            ftrain_la.write(attr.strip()+'\n')
            ftrain_lt.write(line.strip()+'\n')
    print('split done')
    ftrain_st.close()
    ftrain_sa.close()
    ftrain_lt.close()
    ftrain_la.close()
    ftrain_at.close()
    ftrain_aa.close()

In [19]:
split_by_length('train_30.txt','train_30.attr','train_20_s0.txt','train_20_s0.attr','train_20_l0.txt','train_20_l0.attr',split_length=20)
split_by_length('dev_30.txt','dev_30.attr','dev_20_s0.txt','dev_20_s0.attr','dev_20_l0.txt','dev_20_l0.attr',split_length=20)

split done
split done


In [15]:
split_by_length('train_30.txt','train_30.attr','train_20_s.txt','train_20_s.attr','train_20_l.txt','train_20_l.attr',split_length=20)
split_by_length('dev_30.txt','dev_30.attr','dev_20_s.txt','dev_20_s.attr','dev_20_l.txt','dev_20_l.attr',split_length=20)

split done
split done


In [18]:
example=train[:1000]

In [19]:
train[:100]

,stars,text,label
1213589,1.0,Can't believe I just paid 12 dollars for this ...,0.0
239355,1.0,So we came in for Wing night on a Monday and l...,0.0
900328,1.0,"Don't go to this store. The manager is bitch, ...",0.0
135973,1.0,I had used this company in the past & they wer...,0.0
391471,1.0,I am ecstatic that I can finally be closer to ...,0.0
...,...,...,...
38934,1.0,I live only 3 minutes away from this liquor st...,0.0
1098792,1.0,We went to The Shack on a Sunday late afternoo...,0.0
1091788,1.0,"Friday, January 29, 2016, I visited Piece of C...",0.0
865698,1.0,I needed keys made. Pressed service button. Ma...,0.0


In [21]:
lc_e,sen_e,labels_e,dropped,_=write_file(example,'e_text','e_attr')

In [22]:
dropped

865

In [24]:
lc_e

[50,
 104,
 60,
 92,
 109,
 128,
 168,
 180,
 524,
 438,
 482,
 262,
 90,
 158,
 278,
 34,
 84,
 69,
 430,
 89,
 208,
 428,
 196,
 312,
 86,
 65,
 258,
 168,
 558,
 50,
 138,
 60,
 68,
 100,
 65,
 194,
 268,
 164,
 316,
 272,
 17,
 394,
 34,
 61,
 266,
 92,
 378,
 46,
 0,
 230,
 296,
 300,
 412,
 59,
 141,
 310,
 170,
 181,
 356,
 0,
 90,
 56,
 96,
 104,
 280,
 168,
 230,
 0,
 200,
 142,
 386,
 319,
 112,
 0,
 66,
 86,
 61,
 276,
 172,
 84,
 27,
 212,
 710,
 88,
 31,
 86,
 190,
 105,
 422,
 352,
 46,
 165,
 176,
 125,
 60,
 304,
 79,
 40,
 52,
 52,
 101,
 108,
 22,
 214,
 330,
 98,
 108,
 54,
 206,
 522,
 52,
 133,
 0,
 36,
 48,
 204,
 82,
 74,
 298,
 146,
 518,
 122,
 190,
 41,
 71,
 125,
 58,
 55,
 170,
 31,
 276,
 550,
 350,
 72,
 36,
 214,
 107,
 66,
 88,
 43,
 160,
 422,
 102,
 32,
 9,
 184,
 76,
 234,
 216,
 34,
 92,
 154,
 870,
 332,
 53,
 28,
 152,
 226,
 74,
 49,
 84,
 104,
 292,
 24,
 146,
 252,
 264,
 70,
 67,
 132,
 329,
 76,
 14,
 124,
 266,
 242,
 39,
 178,
 0,
 112,
 171

In [25]:
sum(lc_e)/len(lc_e)

182.933

In [26]:
dropped

865

In [27]:
len(example)

1000

In [28]:
sen_e[0]

'cannot believe i just paid 12 dollars for this curry chicken rice .i wish i can compare what they give me with cafe mami  . cannot believe i just paid 12 dollars for this curry chicken rice .i wish i can compare what they give me with cafe mami  .\n'

In [55]:
example['text'].head(1)

962401    Never called me back.  I called 2x with messag...
Name: text, dtype: object

In [352]:
lc_s1,sen_s1,labels_s1,dropped_s1,counts_s1=write_file(dev,'dev_20.txt','dev_20.attr',max_length=20)

In [353]:
lc_s2,sen_s2,labels_s2,dropped_s2,counts_s2=write_file(train,'train_20.txt','train_20.attr',max_length=20)

In [354]:
lc_s3,sen_s3,labels_s3,dropped_s3,counts_s3=write_file(test,'test_20.txt','test_20.attr',max_length=20)

In [345]:
record=dropped_s1/counts_s1,dropped_s2/counts_s2,dropped_s3/counts_s3

In [346]:
record

(0.948965403144166, 0.9492993857007521, 0.949394662155109)